# DICOM Dose and Coordinate handling


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pydicom

In [ ]:
!pip install pydicom pymedphys

import pymedphys
from pymedphys._data import download

In [ ]:
test_dicom_dose_filepath = download.get_file_within_data_zip(
        "dicom_dose_test_data.zip", "RD.wedge.dcm"
    )
ds = pydicom.dcmread(test_dicom_dose_filepath)

## Plot T/C/S dose plans in the IEC Fixed coordinate system:

In [ ]:
def plot_tcs_dose_planes(dose_plane, long_axis, vert_axis, lat_axis, norm_value, dose_index):
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(16, 6), gridspec_kw={'width_ratios': [1, 1, 2, ]})
    fig.suptitle('Dose', fontsize=16)
    
    # Generate transverse dose plot
    axs[0].set_anchor("C")
    axs[0].set_title("Transverse")
    im = axs[0].imshow(
            dose[dose_index[0], :, :],
            cmap="jet",
            vmin=0,
            vmax=max_dose_value,
            aspect="equal",
            extent=(x[0], x[-1], z[-1], z[0]),
        )

    # Generate sagittal dose plot
    axs[1].set_anchor("C")
    axs[1].set_title("Sagittal")
    im = axs[1].imshow(
            np.rot90(dose[:, :, dose_index[2]], k=3),
            cmap="jet",
            vmin=0,
            vmax=max_dose_value,
            aspect="equal",
            extent=(y[0], y[-1], z[-1], z[0]),
        )

    # Generate coronal dose plot
    axs[2].set_anchor("C")
    axs[2].set_title("Coronal")
    im = axs[2].imshow(
            dose[:, dose_index[1], :],
            cmap="jet",
            vmin=0,
            vmax=max_dose_value,
            aspect="equal",
            extent=(x[0], x[-1], y[-1], y[0]),
        )
    fig.colorbar(im, ax=axs[2])

In [ ]:
dose = pymedphys._dicom.dose.dose_from_dataset(ds)

max_dose_value = np.max(dose)
max_dose_index = pymedphys._dicom.coords.unravelled_argmax(dose)
max_dose_x, max_dose_y, max_dose_z = max_dose_index

x, y, z = pymedphys._dicom.coords.xyz_axes_from_dataset(ds, coord_system="IEC FIXED")[0:3]

# For IEC fixed, lateral axis=x, vertical_axis=z, longitudinal_axis=y:

plot_tcs_dose_planes(dose, y, z, x, max_dose_value, max_dose_index)


In [ ]:


test_dicom_dict = {
    "Modality": "RTDOSE",
    "ImagePositionPatient": [-1.0, -1.0, -1.0],
    "ImageOrientationPatient": [1, 0, 0, 0, 1, 0],
    "BitsAllocated": 32,
    "Rows": 2,
    "Columns": 3,
    "NumberOfFrames": 4,
    "PixelRepresentation": 0,
    "SamplesPerPixel": 1,
    "PhotometricInterpretation": "MONOCHROME2",
    "PixelSpacing": [1.0, 1.0],
    "GridFrameOffsetVector": [0.0, 1.0, 2.0, 3.0],
    "PixelData": pixels.tobytes(),
    "DoseGridScaling": 1,
    }

ds = pymedphys._dicom.create.dicom_dataset_from_dict(test_dicom_dict)
ds.fix_meta_info(enforce_standard=False)
ds